# CÀI ĐẶT THƯ VIỆN

In [1]:
! pip install streamlit -q
!pip install -U vnstock3
!pip install mplfinance
!pip install EMD-signal
# Install necessary libraries
!pip install pyemd pandas numpy matplotlib tensorflow scikit-learn
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8

# PORTFOLIO SETTINGS

In [2]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from scipy.optimize import minimize
import streamlit as st
import pandas as pd
import plotly.express as px
from vnstock import Vnstock
from plotly.subplots import make_subplots
from scipy import stats

# ---------------------- Config Page ----------------------
st.set_page_config(
    page_title="Vietnam Stock Portfolio Optimizer",
    page_icon="📈",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ---------------------- Style ----------------------
plt.style.use('dark_background')
st.markdown("""
<style>
    .stApp {
        background-color: #0E1117;
        color: white;
    }
    .sidebar .sidebar-content {
        background-color: #0E1117;
    }
    .stDataFrame {
        background-color: #0E1117;
    }
    table.dataframe {
        background-color: #0E1117;
        color: white !important;
    }
    .dataframe th {
        background-color: #1F2739 !important;
        color: white !important;
    }
    .dataframe td {
        background-color: #0E1117 !important;
        color: white !important;
    }
</style>
""", unsafe_allow_html=True)

# ---------------------- Mock Vietnam Stocks ----------------------
VIETNAM_STOCKS = {
    'HOSE': [
        'VIC', 'VHM', 'VNM', 'VCB', 'CTG', 'ACB', 'MBB', 'VPB', 'TCB', 'SSI',
        'BVH', 'GAS', 'HPG', 'FPT', 'NVL', 'MSN', 'SAB', 'STB', 'PLX', 'PNJ',
        'MWG', 'VRE', 'VJC', 'VPB', 'TPB', 'EIB', 'SHB', 'HDB', 'LPB', 'OCB',
        'TCH', 'KDC', 'DGC', 'PDR', 'REE', 'DXG', 'KDH', 'BCM', 'VCI', 'HAH',
        'GMD', 'HVN', 'VEA', 'POW', 'GVR', 'SBT', 'DPM', 'HSG', 'NKG', 'SCR'
    ],
    'HNX': [
        'PVS', 'NRC', 'VCS', 'SHS', 'BVS', 'ACV', 'HUT', 'IDC', 'MBS', 'VCG',
        'LAS', 'DBC', 'CEO', 'EVS', 'THD', 'VNR', 'VND', 'SVC', 'VMC', 'VAF',
        'VIG', 'VSM', 'VMD', 'VOS', 'SDT', 'TNG', 'TVC', 'TAR', 'HUT', 'ART',
        'HUD', 'HUT', 'DDG', 'DTD', 'DXP', 'DXV', 'DP3', 'DHA', 'DPC', 'DAB'
    ],
    'UPCOM': [
        'QNS', 'VHG', 'DPG', 'KLF', 'LTG', 'VGI', 'VIB', 'SGN', 'BSR', 'DVN',
        'OGC', 'VIX', 'VPD', 'VPO', 'VPS', 'VRC', 'VFG', 'VCF', 'VEF', 'VAF',
        'VHC', 'VHL', 'VIF', 'VIP', 'VKC', 'VKC', 'VLA', 'VLC', 'VLP', 'VLT',
        'VMC', 'VMS', 'VND', 'VNE', 'VNF', 'VNG', 'VNH', 'VNI', 'VNL', 'VNS'
    ]
}

@st.cache_data
def get_vnstock_list():
    stocks = []
    for exchange, tickers in VIETNAM_STOCKS.items():
        for ticker in tickers:
            stocks.append({'Ticker': ticker, 'Exchange': exchange})
    return pd.DataFrame(stocks)

@st.cache_data
def get_stock_data(tickers, start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)
    data = pd.DataFrame(index=date_range)
    for ticker in tickers:
        prices = np.cumprod(1 + np.random.normal(0.001, 0.02, len(date_range)))
        data[ticker] = prices * np.random.randint(10, 100)
    return data

@st.cache_data
def get_financial_data(ticker: str, report_type: str, period: str = 'year') -> pd.DataFrame | None:
    if not ticker or not report_type:
        st.warning("Mã chứng khoán và loại báo cáo không được để trống.")
        return None

    try:
        stock = Vnstock().stock(symbol=ticker, source='TCBS')
        if not hasattr(stock, 'finance'):
            st.error(f"Không tìm thấy dữ liệu tài chính cho mã {ticker}.")
            return None

        df = None

        match report_type:
            case "Balance Sheet":
                df = stock.finance.balance_sheet(period=period)
            case "Income Statement":
                df = stock.finance.income_statement(period=period)
            case "Cash Flow":
                df = stock.finance.cash_flow(period=period)
            case "Financial Ratios":
                df = stock.finance.ratio(period=period, dropna=True)
            case _:
                st.warning(f"Loại báo cáo không hợp lệ: {report_type}")
                return None

        if df is None or df.empty:
            st.warning(f"Không có dữ liệu {report_type} cho mã {ticker}.")
            return None

        # Chuẩn hóa cột thời gian
        if 'report_date' in df.columns:
            df['report_date'] = pd.to_datetime(df['report_date'], errors='coerce')
            df = df.set_index('report_date').sort_index(ascending=False)
        elif 'period' in df.columns:
            df['period'] = pd.to_datetime(df['period'], errors='coerce')
            df = df.set_index('period').sort_index(ascending=False)

        # Ép kiểu dữ liệu quan trọng (nếu cần)
        float_columns = df.select_dtypes(include=['object']).columns
        df[float_columns] = df[float_columns].apply(pd.to_numeric, errors='coerce')

        return df

    except Exception as e:
        st.error(f"Lỗi khi lấy dữ liệu tài chính: {str(e)}")
        st.info("Hãy kiểm tra lại mã chứng khoán, loại báo cáo hoặc kết nối Internet.")
        return None

# ---------------------- Portfolio Functions ----------------------
def get_overview(ticker):
    stock = Vnstock().stock(symbol=ticker, source='TCBS')
    company_info = stock.company.overview().T
    st.subheader(f"🔍 Company overview {ticker}")
    st.dataframe(company_info)

    df = stock.quote.history(start='2023-01-01', end='2024-12-30', interval='1D')
    df.set_index('time', inplace=True)

    # Tính toán MA20 và MA50
    df['MA20'] = df['close'].rolling(window=20).mean()
    df['MA50'] = df['close'].rolling(window=50).mean()

    # Tạo figure với 2 subplots (phần trên: giá + MA, phần dưới: volume)
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=(f"Biểu đồ giá và MA - {ticker}", "Khối lượng giao dịch"),
        row_heights=[0.7, 0.3]
    )

    # 1. Thêm Candlestick vào subplot 1
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Giá'
        ),
        row=1, col=1
    )

    # 2. Thêm MA20 và MA50 vào subplot 1
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['MA20'],
            line=dict(color='orange', width=1.5),
            name='MA20'
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['MA50'],
            line=dict(color='green', width=1.5),
            name='MA50'
        ),
        row=1, col=1
    )

    # 3. Thêm Volume vào subplot 2 (dạng bar chart)
    fig.add_trace(
        go.Bar(
            x=df.index,
            y=df['volume'],
            name='Volume',
            marker_color='blue'
        ),
        row=2, col=1
    )

    # Tùy chỉnh layout
    fig.update_layout(
        height=800,  # Chiều cao tổng thể
        showlegend=True,
        xaxis_rangeslider_visible=False  # Ẩn thanh trượt mặc định
    )

    # Hiển thị biểu đồ trong Streamlit
    st.plotly_chart(fig, use_container_width=True)

    # (Tùy chọn) Giữ lại biểu đồ đường nếu cần
    line_fig = px.line(df, x=df.index, y='close', title=f"Giá đóng cửa (Line Chart) - {ticker}")
    st.plotly_chart(line_fig, use_container_width=True)

def display_financial_tab(selected_tickers):  # Thêm tham số selected_tickers
    """
    Hiển thị tab Financial với các báo cáo tài chính
    Args:
        selected_tickers: Danh sách các mã chứng khoán đã chọn
    """
    st.subheader("📊 Financial Statements & Ratios")

    if not selected_tickers:
        st.warning("Vui lòng chọn ít nhất một mã chứng khoán trong sidebar")
        return

    # Tạo các tab con trong tab Financial
    tab1, tab2, tab3, tab4 = st.tabs([
        "📜 Balance Sheet",
        "💰 Income Statement",
        "💵 Cash Flow",
        "📈 Financial Ratios"
    ])

    selected_ticker = st.selectbox(
        "Select a stock ticker",
        selected_tickers,
        key="financial_ticker"
    )

    period = st.radio(
        "Reporting period",
        ["Yearly"],
        index=0,
        horizontal=True,
        key="financial_period"
    )
    period = 'year' if period == "Yearly" else 'quarter'

    with tab1:
        st.markdown(f"### 🏦 Balance Sheet - {selected_ticker}")
        df = get_financial_data(selected_ticker, "Balance Sheet", period)
        if df is not None:
            st.dataframe(df.style.format("{:,.0f}"), use_container_width=True)
        else:
            st.warning("No balance sheet data available")

    with tab2:
        st.markdown(f"### 💰 Income Statement - {selected_ticker}")
        df = get_financial_data(selected_ticker, "Income Statement", period)
        if df is not None:
            st.dataframe(df.style.format("{:,.0f}"), use_container_width=True)
        else:
            st.warning("No income statement data available")

    with tab3:
        st.markdown(f"### 💵 Cash Flow Statement - {selected_ticker}")
        df = get_financial_data(selected_ticker, "Cash Flow", period)
        if df is not None:
            st.dataframe(df.style.format("{:,.0f}"), use_container_width=True)
        else:
            st.warning("No cash flow data available")

    with tab4:
        st.markdown(f"### 📈 Financial Ratios - {selected_ticker}")
        df = get_financial_data(selected_ticker, "Financial Ratios", period)
        if df is not None:
            st.dataframe(df.style.format("{:,.2f}"), use_container_width=True)

            if not df.empty:
                important_ratios = ['roe', 'roa', 'gross_margin', 'net_interest_margin', 'debt_to_equity']
                available_ratios = [ratio for ratio in important_ratios if ratio in df.columns]

                if available_ratios:
                    st.markdown("#### Key Ratios Trend")
                    fig = go.Figure()

                    for ratio in available_ratios:
                        fig.add_trace(go.Scatter(
                            x=df.index,
                            y=df[ratio],
                            mode='lines+markers',
                            name=ratio.upper()
                        ))

                    fig.update_layout(
                        xaxis_title='Period',
                        yaxis_title='Ratio Value',
                        hovermode='x unified'
                    )
                    st.plotly_chart(fig, use_container_width=True)
        else:
            st.warning("No financial ratios data available")

def calculate_returns(df):
    return np.log(1 + df.pct_change())

def portfolio_annualized_return(weights, mean_returns):
    return np.sum(mean_returns * weights) * 252

def portfolio_annualized_volatility(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)

def generate_random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((4, num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        weights_record.append(weights)

        ret = portfolio_annualized_return(weights, mean_returns)
        vol = portfolio_annualized_volatility(weights, cov_matrix)
        sharpe = (ret - risk_free_rate) / vol
        sortino = (ret - risk_free_rate) / np.std(np.minimum(0, ret))

        results[0,i] = ret
        results[1,i] = vol
        results[2,i] = sharpe
        results[3,i] = sortino
    return results, weights_record

def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_ret = portfolio_annualized_return(weights, mean_returns)
    p_vol = portfolio_annualized_volatility(weights, cov_matrix)
    return -(p_ret - risk_free_rate) / p_vol

def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0, 1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = minimize(neg_sharpe_ratio, num_assets*[1./num_assets], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def min_variance(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0, 1.0)
    bounds = tuple(bound for asset in range(num_assets))

    def portfolio_volatility(weights, cov_matrix):
        return portfolio_annualized_volatility(weights, cov_matrix)

    result = minimize(portfolio_volatility, num_assets*[1./num_assets], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def efficient_return(mean_returns, cov_matrix, target):
    num_assets = len(mean_returns)
    args = (cov_matrix)

    def portfolio_return(weights):
        return portfolio_annualized_return(weights, mean_returns)

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_return(x) - target},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))

    def portfolio_volatility(weights, cov_matrix):
        return portfolio_annualized_volatility(weights, cov_matrix)

    result = minimize(portfolio_volatility, num_assets*[1./num_assets], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def efficient_frontier(mean_returns, cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))
    return efficients

# ====================== Monte Carlo Simulation Function (Normalized) ======================
def run_monte_carlo_forecast(price_data, num_simulations=10000, forecast_days=30):
    """
    Fixed version with proper return values
    """
    returns = np.log(price_data['close'] / price_data['close'].shift(1)).dropna()

    mu = returns.mean()
    sigma = returns.std()
    last_price = price_data['close'].iloc[-1]

    simulations = np.zeros((num_simulations, forecast_days + 1))
    simulations[:, 0] = last_price  # Normalized starting price

    # Vectorized computation
    for d in range(1, forecast_days + 1):
        shocks = np.random.normal(mu, sigma, num_simulations)
        simulations[:, d] = simulations[:, d-1] * np.exp(shocks)

    mean_forecast = simulations.mean(axis=0)
    ci_90_lower = np.percentile(simulations, 5, axis=0)
    ci_90_upper = np.percentile(simulations, 95, axis=0)
    ci_50_lower = np.percentile(simulations, 25, axis=0)
    ci_50_upper = np.percentile(simulations, 75, axis=0)

    return {
        'mean': mean_forecast,
        'ci_90_lower': ci_90_lower,
        'ci_90_upper': ci_90_upper,
        'ci_50_lower': ci_50_lower,
        'ci_50_upper': ci_50_upper,
        'paths': simulations[:100], # Return first 100 paths
        'last_price': last_price
    }

# ====================== VaR and CVaR Calculation Functions ======================
def calculate_historical_var_cvar(portfolio_returns, confidence_level=0.95):
    """
    Calculates 1-day Historical VaR and CVaR.

    Args:
        portfolio_returns (pd.Series): Series of historical daily portfolio returns.
        confidence_level (float): The confidence level (e.g., 0.95 for 95%).

    Returns:
        tuple: (var_percent, cvar_percent)
               VaR and CVaR as negative percentages (representing loss).
    """
    if portfolio_returns.empty:
        return np.nan, np.nan
    # VaR is the percentile of the historical returns distribution
    var_percent = portfolio_returns.quantile(1 - confidence_level)
    # CVaR is the average of returns less than or equal to the VaR
    cvar_percent = portfolio_returns[portfolio_returns <= var_percent].mean()
    # Return as negative numbers representing loss potential
    return var_percent, cvar_percent

def calculate_parametric_var_cvar(portfolio_returns, confidence_level=0.95):
    """
    Calculates 1-day Parametric (Variance-Covariance) VaR and CVaR assuming normality.

    Args:
        portfolio_returns (pd.Series): Series of historical daily portfolio returns.
        confidence_level (float): The confidence level (e.g., 0.95 for 95%).

    Returns:
        tuple: (var_percent, cvar_percent)
               VaR and CVaR as negative percentages (representing loss).
    """
    if portfolio_returns.empty:
        return np.nan, np.nan

    # Calculate mean and standard deviation of portfolio returns
    mu = portfolio_returns.mean()
    sigma = portfolio_returns.std()

    # Z-score for the given confidence level
    z_score = stats.norm.ppf(1 - confidence_level) # This will be negative for typical confidence levels

    # Calculate VaR (often mu is assumed 0 for 1-day VaR, but we include it here)
    var_percent = mu + z_score * sigma

    # Calculate CVaR (Expected Shortfall) for normal distribution
    # CVaR = mu - sigma * (pdf(Z) / (1 - cdf(Z)))
    # where Z is the ppf(confidence_level), and (1 - cdf(Z)) is alpha = 1 - confidence_level
    alpha = 1 - confidence_level
    z_score_cvar = stats.norm.ppf(alpha) # = z_score in our case
    cvar_percent = mu - sigma * (stats.norm.pdf(z_score_cvar) / alpha)

    return var_percent, cvar_percent

# ---------------------- Main App ----------------------
def main():
    st.title("🇻🇳 Vietnam Stock Portfolio Optimizer")
    st.markdown("""
    <style>
    .big-font {
        font-size:16px !important;
        color: #FF4B4B;
    }
    </style>
    <p class="big-font">Optimize your Vietnam stock portfolio using Modern Portfolio Theory</p>
    """, unsafe_allow_html=True)

    with st.sidebar:
        st.header("Portfolio Configuration")
        col1, col2 = st.columns(2)
        with col1:
            start_date = st.date_input("Start date", datetime(2022, 1, 1))
        with col2:
            end_date = st.date_input("End date", datetime.now())

        st.subheader("Stock Selection")
        listing_df = get_vnstock_list()
        selected_exchanges = st.multiselect("Filter by exchange", options=listing_df['Exchange'].unique(), default=['HOSE'])
        if selected_exchanges:
            listing_df = listing_df[listing_df['Exchange'].isin(selected_exchanges)]

        selected_tickers = st.multiselect("Select stocks", options=listing_df['Ticker'].unique(), default=['TCB', 'STB', 'ACB', 'HAH', 'GMD', 'HPG', 'VIC', 'VHM', 'MWG', 'MSN'])

        st.subheader("Optimization Parameters")
        num_portfolios = st.slider("Number of portfolios to generate", 1000, 20000, 5000, step=1000)
        risk_free_rate = st.slider("Risk-free rate (%)", 0.0, 10.0, 5.0) / 100

    if not selected_tickers:
        st.warning("Please select at least one stock")
        return

    price_data = get_stock_data(selected_tickers, start_date, end_date)
    returns = calculate_returns(price_data).dropna()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    volatility = returns.std()
    results, weights = generate_random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate)
    max_sharpe_idx = np.argmax(results[2])
    min_vol_idx = np.argmin(results[1])

    # Calculate optimized portfolios
    max_sharpe_weights = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    min_vol_weights = min_variance(mean_returns, cov_matrix)

    # Efficient Frontier calculation
    ret_range = np.linspace(results[0,:].min(), results[0,:].max(), 20)
    efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, ret_range)

    # Define the main tabs
    # Trong phần định nghĩa các tab, thêm tab mới vào danh sách
    tab_overview, tab_financial, tab_monte_carlo, tab_var_stock, tab_portfolio = st.tabs([
        "🏢 Overview", "📊 Financial", "📈 Monte Carlo", "💵 Value At Risk", "📈 My Portfolio"
    ])

    with tab_overview:
        st.subheader("🏢 Company Overview")
        overview_ticker = st.selectbox("Select a stock for overview", options=selected_tickers, index=0)
        if overview_ticker:
            get_overview(overview_ticker)

    with tab_financial:
        display_financial_tab(selected_tickers)

    with tab_var_stock:
        st.subheader("💵 Value at Risk (VaR) & Conditional VaR (CVaR) Analysis for Individual Stock")
        st.markdown("""
                Phân tích rủi ro cho cổ phiếu đơn lẻ sử dụng Value at Risk (VaR) và Conditional VaR (CVaR)
                """)

        # Chọn cổ phiếu
        selected_stock_var = st.selectbox(
                "Select stock for analysis",
                options=selected_tickers,
                index=0,
                key="var_stock_selector"
        )

        if selected_stock_var:
                # Input parameters
                col1_var, col2_var = st.columns(2)
                with col1_var:
                        confidence_level_var = st.slider(
                                "Confidence Level (%)",
                                min_value=90.0,
                                max_value=99.9,
                                value=95.0,
                                step=0.5,
                                format="%.1f%%",
                                key="stock_var_conf_level"
                        ) / 100

                with col2_var:
                        investment_value = st.number_input(
                                "Investment Value (VND)",
                                min_value=1000000,
                                value=1000000000,
                                step=10000000,
                                format="%d",
                                key="stock_investment_val"
                        )

                # Lấy dữ liệu returns
                stock_returns = returns[selected_stock_var].dropna()

                # Tính toán VaR/CVaR
                hist_var, hist_cvar = calculate_historical_var_cvar(stock_returns, confidence_level_var)
                para_var, para_cvar = calculate_parametric_var_cvar(stock_returns, confidence_level_var)

                # Hiển thị kết quả
                st.markdown(f"#### 📊 1-Day VaR & CVaR Results for {selected_stock_var}")

                # Tạo columns cho metrics
                col_metric1, col_metric2 = st.columns(2)

                with col_metric1:
                        st.metric("Historical VaR",
                                        f"{-hist_var*100:.2f}%",
                                        delta=f"{-hist_var*investment_value:,.0f} VND",
                                        delta_color="inverse")

                        st.metric("Historical CVaR",
                                        f"{-hist_cvar*100:.2f}%",
                                        delta=f"{-hist_cvar*investment_value:,.0f} VND",
                                        delta_color="inverse")

                with col_metric2:
                        st.metric("Parametric VaR",
                                        f"{-para_var*100:.2f}%",
                                        delta=f"{-para_var*investment_value:,.0f} VND",
                                        delta_color="inverse")

                        st.metric("Parametric CVaR",
                                        f"{-para_cvar*100:.2f}%",
                                        delta=f"{-para_cvar*investment_value:,.0f} VND",
                                        delta_color="inverse")

                # Vẽ biểu đồ phân phối
                fig = px.histogram(stock_returns*100,
                                                nbins=50,
                                                title=f"Distribution of Daily Returns - {selected_stock_var}",
                                                labels={'value':'Daily Return (%)'})

                # Thêm các đường VaR
                fig.add_vline(x=hist_var*100, line_dash="dash", line_color="red",
                                        annotation_text=f"Historical VaR: {hist_var*100:.2f}%")
                fig.add_vline(x=para_var*100, line_dash="dash", line_color="green",
                                        annotation_text=f"Parametric VaR: {para_var*100:.2f}%")

                fig.update_layout(bargap=0.01,
                                          xaxis_tickformat=".2f%",
                                          showlegend=False)

                st.plotly_chart(fig, use_container_width=True)

                # Hiển thị cảnh báo rủi ro
                st.markdown("""
                        **📌 Risk Interpretation:**
                        - *Historical VaR*: Dựa trên phân phối lợi nhuận lịch sử
                        - *Parametric VaR*: Giả định phân phối chuẩn
                        - *CVaR*: Mức lỗ trung bình trong các trường hợp xấu nhất
                        """)

    with tab_portfolio:
        subtab_prices, subtab_optimization, subtab_correlation, subtab_cumulative, subtab_drawdown, subtab_efficient_frontier, subtab_risk_analysis, subtab_rolling_metrics, subtab_return_analysis, subtab_var_portfolio = st.tabs([
                "📈 Prices", "⚙️ Optimization", "📊 Correlation", "📉 Cumulative", "🛡️ Drawdown", "🎯 Efficient Frontier", "📊 Risk Analysis", "📅 Rolling Metrics", "📊 Return Analysis", "🛡️ Portfolio VaR"
            ])

        with subtab_prices:
            st.subheader("Stock Price Movement")
            fig = px.line(price_data, x=price_data.index, y=price_data.columns, title="Price Trends")
            st.plotly_chart(fig, use_container_width=True)

        with subtab_optimization:
            st.subheader("Portfolio Optimization Results")

            # Display key metrics
            col1, col2, col3 = st.columns(3)
            col1.metric("Max Sharpe Ratio (Random)", f"{results[2,max_sharpe_idx]:.2f}")
            col2.metric("Min Volatility (Random)", f"{results[1,min_vol_idx]:.2f}")
            col3.metric("Risk-Free Rate", f"{risk_free_rate:.2%}")

            # Portfolio allocations
            st.subheader("Optimal Portfolio Allocations")

            # Random portfolios
            cols = st.columns(2)
            with cols[0]:
                st.markdown("### Max Sharpe (Random)")
                weights_df = pd.DataFrame({
                    'Stock': selected_tickers,
                    'Weight': weights[max_sharpe_idx],
                    'Allocation (%)': weights[max_sharpe_idx] * 100
                })
                st.dataframe(weights_df.style.format({'Weight': '{:.4f}', 'Allocation (%)': '{:.2f}%'}))

                fig_pie = px.pie(weights_df, values='Allocation (%)', names='Stock',
                                title="Max Sharpe Allocation (Random)")
                st.plotly_chart(fig_pie, use_container_width=True)

            with cols[1]:
                st.markdown("### Min Volatility (Random)")
                weights_df = pd.DataFrame({
                    'Stock': selected_tickers,
                    'Weight': weights[min_vol_idx],
                    'Allocation (%)': weights[min_vol_idx] * 100
                })
                st.dataframe(weights_df.style.format({'Weight': '{:.4f}', 'Allocation (%)': '{:.2f}%'}))

                fig_pie = px.pie(weights_df, values='Allocation (%)', names='Stock',
                                title="Min Volatility Allocation (Random)")
                st.plotly_chart(fig_pie, use_container_width=True)

            # Optimized portfolios
            st.subheader("Mathematically Optimized Portfolios")
            cols = st.columns(2)
            with cols[0]:
                st.markdown("### Optimized Max Sharpe")
                opt_weights_df = pd.DataFrame({
                    'Stock': selected_tickers,
                    'Weight': max_sharpe_weights,
                    'Allocation (%)': max_sharpe_weights * 100
                })
                st.dataframe(opt_weights_df.style.format({'Weight': '{:.4f}', 'Allocation (%)': '{:.2f}%'}))

                fig_pie = px.pie(opt_weights_df, values='Allocation (%)', names='Stock',
                                title="Optimized Max Sharpe Allocation")
                st.plotly_chart(fig_pie, use_container_width=True)

            with cols[1]:
                st.markdown("### Optimized Min Volatility")
                opt_weights_df = pd.DataFrame({
                    'Stock': selected_tickers,
                    'Weight': min_vol_weights,
                    'Allocation (%)': min_vol_weights * 100
                })
                st.dataframe(opt_weights_df.style.format({'Weight': '{:.4f}', 'Allocation (%)': '{:.2f}%'}))

                fig_pie = px.pie(opt_weights_df, values='Allocation (%)', names='Stock',
                                title="Optimized Min Volatility Allocation")
                st.plotly_chart(fig_pie, use_container_width=True)

        with subtab_var_portfolio:
            st.subheader("💵 Value at Risk (VaR) & Conditional VaR (CVaR) Analysis")
        # [Toàn bộ nội dung của tab_var_portfolio original được đặt ở đây]
            st.markdown("""
        Value at Risk (VaR) ước tính mức lỗ tiềm năng tối đa trong một khoảng thời gian nhất định, với một mức độ tin cậy cụ thể, trong điều kiện thị trường bình thường.
        Conditional VaR (CVaR), hay Expected Shortfall (ES), đo lường mức lỗ *trung bình* dự kiến nếu khoản lỗ đó *vượt quá* mức VaR. CVaR được coi là thước đo rủi ro tốt hơn VaR vì nó nắm bắt rủi ro đuôi (tail risk).
        """)

        # --- User Inputs ---
        col1_var, col2_var, col3_var = st.columns(3)
        with col1_var:
            portfolio_choice_var = st.selectbox(
                "Select Portfolio for Analysis",
                options=["Optimized Max Sharpe", "Optimized Min Volatility"],
                key="portfolio_choice_var"
            )
        with col2_var:
            confidence_level_var = st.slider(
                "Confidence Level (%)",
                min_value=90.0,
                max_value=99.9,
                value=95.0,
                step=0.5,
                format="%.1f%%",
                key="confidence_level_var"
            ) / 100
        with col3_var:
            portfolio_value_var = st.number_input(
                "Current Portfolio Value",
                min_value=1.0,
                value=1000000.0,
                step=10000.0,
                format="%f",
                key="portfolio_value_var"
            )

        # --- Calculations ---
        if portfolio_choice_var and portfolio_value_var > 0:
            if portfolio_choice_var == "Optimized Max Sharpe":
                weights_var = max_sharpe_weights
            else:
                weights_var = min_vol_weights

            if 'returns' in locals() and weights_var is not None and len(returns.columns) == len(weights_var):
                portfolio_returns_var = returns.dot(weights_var)

                hist_var, hist_cvar = calculate_historical_var_cvar(portfolio_returns_var, confidence_level_var)
                para_var, para_cvar = calculate_parametric_var_cvar(portfolio_returns_var, confidence_level_var)

                st.markdown(f"#### 1-Day VaR & CVaR Results for {portfolio_choice_var} Portfolio")
                st.markdown(f"Confidence Level: **{confidence_level_var:.1%}** | Portfolio Value: **{portfolio_value_var:,.0f}**")

                col1_res, col2_res = st.columns(2)

                with col1_res:
                    st.markdown("**Historical Method**")
                    if not np.isnan(hist_var):
                        st.metric(
                            label=f"Historical VaR ({1-confidence_level_var:.1%})",
                            value=f"{hist_var:.2%}",
                            delta=f"{hist_var * portfolio_value_var:,.0f} (potential loss amount)",
                            delta_color="inverse"
                        )
                    else:
                        st.warning("Could not calculate Historical VaR.")

                    if not np.isnan(hist_cvar):
                        st.metric(
                            label=f"Historical CVaR (Expected Shortfall if loss > VaR)",
                            value=f"{hist_cvar:.2%}",
                            delta=f"{hist_cvar * portfolio_value_var:,.0f} (avg. loss in worst cases)",
                            delta_color="inverse"
                        )
                    else:
                        st.warning("Could not calculate Historical CVaR.")

                with col2_res:
                    st.markdown("**Parametric Method (Normal Distribution)**")
                    if not np.isnan(para_var):
                        st.metric(
                            label=f"Parametric VaR ({1-confidence_level_var:.1%})",
                            value=f"{para_var:.2%}",
                            delta=f"{para_var * portfolio_value_var:,.0f} (potential loss amount)",
                            delta_color="inverse"
                        )
                    else:
                        st.warning("Could not calculate Parametric VaR.")

                    if not np.isnan(para_cvar):
                        st.metric(
                            label=f"Parametric CVaR (Expected Shortfall if loss > VaR)",
                            value=f"{para_cvar:.2%}",
                            delta=f"{para_cvar * portfolio_value_var:,.0f} (avg. loss in worst cases)",
                            delta_color="inverse"
                        )
                    else:
                        st.warning("Could not calculate Parametric CVaR.")

                st.markdown("#### Historical Daily Portfolio Returns Distribution")
                fig_hist_var = px.histogram(
                    portfolio_returns_var,
                    nbins=100,
                    title=f"Distribution of Daily Returns - {portfolio_choice_var}"
                )
                fig_hist_var.update_layout(
                    xaxis_title="Daily Return",
                    yaxis_title="Frequency",
                    xaxis_tickformat='.2%'
                )
                if not np.isnan(hist_var):
                    fig_hist_var.add_vline(x=hist_var, line_dash="dash", line_color="red", annotation_text=f"Hist VaR {hist_var:.2%}", annotation_position="top left")
                if not np.isnan(para_var):
                    fig_hist_var.add_vline(x=para_var, line_dash="dash", line_color="orange", annotation_text=f"Para VaR {para_var:.2%}", annotation_position="bottom left")

                st.plotly_chart(fig_hist_var, use_container_width=True)

            else:
                st.error("Could not calculate portfolio returns. Ensure stocks are selected and weights are calculated.")
        else:
            st.info("Select a portfolio, set confidence level, and enter portfolio value.")

        with subtab_correlation:
            st.subheader("Correlation Heatmap")
            fig_corr = px.imshow(returns.corr(), text_auto=True, color_continuous_scale='RdBu_r')
            st.plotly_chart(fig_corr, use_container_width=True)

        with subtab_cumulative:
            st.subheader("Cumulative Returns")
            cumulative_returns = (1 + returns).cumprod()
            fig_cum = px.line(cumulative_returns, x=cumulative_returns.index, y=cumulative_returns.columns,
                             title="Cumulative Returns")
            fig_cum.update_layout(yaxis_title="Cumulative Return", xaxis_title="Date")
            st.plotly_chart(fig_cum, use_container_width=True)

        with subtab_drawdown:
            st.subheader("Drawdown Analysis")
            drawdown = (1 + returns).cumprod() / (1 + returns).cumprod().cummax() - 1
            fig_drawdown = px.line(drawdown, x=drawdown.index, y=drawdown.columns,
                                  title="Drawdown from Peak")
            fig_drawdown.update_layout(yaxis_tickformat='.0%', yaxis_title="Drawdown", xaxis_title="Date")
            st.plotly_chart(fig_drawdown, use_container_width=True)

        with subtab_efficient_frontier:
            st.subheader("Efficient Frontier")

            # Create efficient frontier plot
            fig = go.Figure()

            # Add random portfolios
            fig.add_trace(go.Scatter(
                x=results[1,:], y=results[0,:],
                mode='markers',
                marker=dict(color=results[2,:], colorscale='Viridis', size=5, showscale=True),
                name='Random Portfolios'
            ))

            # Add optimal portfolios
            fig.add_trace(go.Scatter(
                x=[results[1,max_sharpe_idx]], y=[results[0,max_sharpe_idx]],
                mode='markers',
                marker=dict(color='gold', size=12),
                name='Max Sharpe (Random)'
            ))

            fig.add_trace(go.Scatter(
                x=[results[1,min_vol_idx]], y=[results[0,min_vol_idx]],
                mode='markers',
                marker=dict(color='red', size=12),
                name='Min Volatility (Random)'
            ))

            # Add optimized portfolios
            max_sharpe_vol = portfolio_annualized_volatility(max_sharpe_weights, cov_matrix)
            max_sharpe_ret = portfolio_annualized_return(max_sharpe_weights, mean_returns)

            fig.add_trace(go.Scatter(
                x=[max_sharpe_vol], y=[max_sharpe_ret],
                mode='markers',
                marker=dict(color='green', size=15, symbol='star'),
                name='Optimized Max Sharpe'
            ))

            min_vol_vol = portfolio_annualized_volatility(min_vol_weights, cov_matrix)
            min_vol_ret = portfolio_annualized_return(min_vol_weights, mean_returns)

            fig.add_trace(go.Scatter(
                x=[min_vol_vol], y=[min_vol_ret],
                mode='markers',
                marker=dict(color='blue', size=15, symbol='star'),
                name='Optimized Min Volatility'
            ))

            # Add efficient frontier line
            eff_vols = [portfolio_annualized_volatility(w, cov_matrix) for w in efficient_portfolios]
            fig.add_trace(go.Scatter(
                x=eff_vols, y=ret_range,
                mode='lines',
                line=dict(color='white', width=2, dash='dash'),
                name='Efficient Frontier'
            ))

            fig.update_layout(
                xaxis_title='Annualized Volatility',
                yaxis_title='Annualized Return',
                title='Efficient Frontier with Optimal Portfolios',
                hovermode='x unified'
            )
            st.plotly_chart(fig, use_container_width=True)

        with subtab_risk_analysis:
            st.subheader("Risk Analysis")

            # Calculate individual stock metrics
            stock_vols = np.sqrt(np.diag(cov_matrix)) * np.sqrt(252)
            stock_returns = mean_returns * 252

            # Create risk-return plot
            fig = go.Figure()

            # Add individual stocks
            fig.add_trace(go.Scatter(
                x=stock_vols,
                y=stock_returns,
                mode='markers+text',
                marker=dict(size=12, color='orange'),
                text=selected_tickers,
                textposition='top center',
                name='Individual Stocks'
            ))

            # Add portfolios
            fig.add_trace(go.Scatter(
                x=[results[1,max_sharpe_idx], max_sharpe_vol, results[1,min_vol_idx], min_vol_vol],
                y=[results[0,max_sharpe_idx], max_sharpe_ret, results[0,min_vol_idx], min_vol_ret],
                mode='markers',
                marker=dict(size=15, color=['gold', 'green', 'red', 'blue'],
                           symbol=['circle', 'star', 'circle', 'star']),
                text=['Random Max Sharpe', 'Optimized Max Sharpe', 'Random Min Vol', 'Optimized Min Vol'],
                name='Portfolios'
            ))

            fig.update_layout(
                xaxis_title='Annualized Volatility',
                yaxis_title='Annualized Return',
                title='Risk-Return Profile: Stocks vs Portfolios'
            )
            st.plotly_chart(fig, use_container_width=True)

        with subtab_rolling_metrics:
            st.subheader("Rolling Metrics")

            window = st.slider("Rolling window (days)", 30, 252, 90)

            # Calculate rolling metrics
            rolling_vol = returns.rolling(window).std() * np.sqrt(252)
            rolling_sharpe = returns.rolling(window).mean() / returns.rolling(window).std() * np.sqrt(252)

            # Plot rolling volatility
            st.markdown("#### Rolling Annualized Volatility")
            fig_vol = px.line(rolling_vol, x=rolling_vol.index, y=rolling_vol.columns,
                             title=f"{window}-Day Rolling Volatility")
            fig_vol.update_layout(yaxis_tickformat='.0%')
            st.plotly_chart(fig_vol, use_container_width=True)

            # Plot rolling Sharpe ratio
            st.markdown("#### Rolling Sharpe Ratio")
            fig_sharpe = px.line(rolling_sharpe, x=rolling_sharpe.index, y=rolling_sharpe.columns,
                                title=f"{window}-Day Rolling Sharpe Ratio")
            st.plotly_chart(fig_sharpe, use_container_width=True)
        with subtab_return_analysis:
            st.subheader("Portfolio Return Analysis")

            # Chọn loại danh mục
            portfolio_type = st.selectbox(
                "Select Portfolio Type",
                ["Max Sharpe", "Min Volatility"],
                index=0,
                key="portfolio_type_analysis"
            )

            # Lấy weights tương ứng
            if portfolio_type == "Max Sharpe":
                weights = max_sharpe_weights
            else:
                weights = min_vol_weights

            # --- PHẦN BỔ SUNG: HIỂN THỊ GIÁ TRỊ ĐẦU TƯ VÀ LỢI NHUẬN KỲ VỌNG ---
            initial_investment = 100000000  # 100 triệu VND
            annual_return = portfolio_annualized_return(weights, mean_returns)
            expected_value = initial_investment * (1 + annual_return)

            # Tạo metrics
            col1, col2, col3 = st.columns(3)
            col1.metric("Initial Investment", f"{initial_investment:,.0f} VND")
            col2.metric("Expected Annual Return", f"{annual_return:.2%}")
            col3.metric("Expected Value After 1 Year",
                        f"{expected_value:,.0f} VND",
                        delta=f"{expected_value - initial_investment:,.0f} VND")

            # Tính toán lợi nhuận danh mục chính xác
            portfolio_daily_returns = returns.dot(weights)
            portfolio_cumulative_returns = (1 + portfolio_daily_returns).cumprod()

            # Tính toán chỉ số thị trường
            market_tickers = ['VNINDEX', 'VN30']
            market_data = get_stock_data(market_tickers, start_date, end_date)
            market_returns = calculate_returns(market_data)

            # Chuẩn hóa tất cả về base 100 triệu VND để dễ so sánh
            base_value = initial_investment / 1000000  # Chuẩn hóa về đơn vị triệu VND
            normalized_portfolio = portfolio_cumulative_returns * base_value
            normalized_market = (market_data / market_data.iloc[0]) * base_value

            # Tạo DataFrame tổng hợp
            comparison_df = pd.DataFrame({
                'Your Portfolio': normalized_portfolio,
                'VNINDEX': normalized_market['VNINDEX'],
                'VN30': normalized_market['VN30']
            })

            # Vẽ biểu đồ ----------------------------------------------------------------
            fig = px.line(comparison_df,
                            title=f"Portfolio Performance vs Market Indices (Base: {base_value:,.0f} million VND)",
                            labels={'value': 'Portfolio Value (million VND)'})

            # Thêm annotation cho giá trị cuối kỳ
            for column in comparison_df.columns:
                last_value = comparison_df[column].iloc[-1]
                fig.add_annotation(
                    x=comparison_df.index[-1],
                    y=last_value,
                    text=f"{column}: {last_value:,.1f}",
                    showarrow=True,
                    arrowhead=1,
                    yshift=10 if column == 'Your Portfolio' else -10
                )

            fig.update_traces(
                line=dict(width=3),
                selector={'name': 'Your Portfolio'}
            )

            fig.update_layout(
                hovermode='x unified',
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                )
            )

            st.plotly_chart(fig, use_container_width=True)

            # --- PHẦN BỔ SUNG: BẢNG SO SÁNH HIỆU SUẤT CHI TIẾT ---
            st.markdown("#### Detailed Performance Comparison")

            # Tính các chỉ số quan trọng
            performance_metrics = {
                'Initial Value (million VND)': [base_value] * 3,
                'Final Value (million VND)': [
                    comparison_df['Your Portfolio'].iloc[-1],
                    comparison_df['VNINDEX'].iloc[-1],
                    comparison_df['VN30'].iloc[-1]
                ],
                'Total Return (%)': [
                    (comparison_df['Your Portfolio'].iloc[-1]/base_value - 1)*100,
                    (comparison_df['VNINDEX'].iloc[-1]/base_value - 1)*100,
                    (comparison_df['VN30'].iloc[-1]/base_value - 1)*100
                ],
                'Annualized Return (%)': [
                    annual_return * 100,
                    market_returns['VNINDEX'].mean() * 252 * 100,
                    market_returns['VN30'].mean() * 252 * 100
                ],
                'Annualized Volatility (%)': [
                    portfolio_annualized_volatility(weights, cov_matrix) * 100,
                    market_returns['VNINDEX'].std() * np.sqrt(252) * 100,
                    market_returns['VN30'].std() * np.sqrt(252) * 100
                ]
            }

            df_performance = pd.DataFrame(
                performance_metrics,
                index=['Your Portfolio', 'VNINDEX', 'VN30']
            )

            st.dataframe(
                df_performance.style.format({
                    'Initial Value (million VND)': '{:,.1f}',
                    'Final Value (million VND)': '{:,.1f}',
                    'Total Return (%)': '{:,.2f}%',
                    'Annualized Return (%)': '{:,.2f}%',
                    'Annualized Volatility (%)': '{:,.2f}%'
                }),
                use_container_width=True
            )

            # Phân tích rủi ro bổ sung
            st.markdown("#### Risk-Return Comparison")

            # Vẽ biểu đồ risk-return
            fig = go.Figure()

            fig.add_trace(go.Scatter(
                x=[df_performance.loc['VNINDEX', 'Annualized Volatility (%)']],
                y=[df_performance.loc['VNINDEX', 'Annualized Return (%)']],
                mode='markers+text',
                text=['VNINDEX'],
                marker=dict(size=15, color='blue'),
                name='Market Indices'
            ))

            fig.add_trace(go.Scatter(
                x=[df_performance.loc['VN30', 'Annualized Volatility (%)']],
                y=[df_performance.loc['VN30', 'Annualized Return (%)']],
                mode='markers+text',
                text=['VN30'],
                marker=dict(size=15, color='green'),
                name='Market Indices'
            ))

            fig.add_trace(go.Scatter(
                x=[df_performance.loc['Your Portfolio', 'Annualized Volatility (%)']],
                y=[df_performance.loc['Your Portfolio', 'Annualized Return (%)']],
                mode='markers+text',
                text=['Your Portfolio'],
                marker=dict(
                    size=20,
                    color='red',
                    symbol='star'
                ),
                name='Portfolio'
            ))

            fig.update_layout(
                xaxis_title="Annualized Volatility (%)",
                yaxis_title="Annualized Return (%)",
                title="Risk-Return Profile Comparison",
                template='plotly_white'
            )

            st.plotly_chart(fig, use_container_width=True)
    with tab_monte_carlo:
        st.subheader("📈 Monte Carlo Simulation")

    # Lấy dữ liệu từ session state
        mc_stock = st.selectbox(
            "Select stock for forecast",
            options=selected_tickers,
            index=0,
            key="mc_stock_final"
        )

        # Khởi tạo dữ liệu giá
        if f'monte_data_{mc_stock}' not in st.session_state:
            stock = Vnstock().stock(symbol=mc_stock, source='TCBS')
            df = stock.quote.history(start='2023-01-01', end='2024-12-30', interval='1D')
            df.set_index('time', inplace=True)
            st.session_state[f'monte_data_{mc_stock}'] = df

        price_data = st.session_state[f'monte_data_{mc_stock}']

# Thêm slider chọn số lượng simulations
        col1, col2 = st.columns(2)
        with col1:
            num_simulations_mc = st.slider(
                "Number of simulations",
                min_value=500,
                max_value=20000,
                value=5000,
                step=500,
                key="num_simulations_mc_custom"
            )
        with col2:
            forecast_days = st.slider(
                "Number of paths to display",
                min_value=10,
                max_value=200,
                value=100,
                step=10,
                key="num_paths_to_show"
            )



        # Chạy mô phỏng
        result = run_monte_carlo_forecast(price_data, num_simulations_mc, forecast_days)

        # Vẽ biểu đồ
        fig = go.Figure()

# 1. Hiển thị số lượng đường forecast được chọn

        for path in result['paths']:
            fig.add_trace(go.Scatter(
                    x=list(range(forecast_days+1)),
                    y=path,
                    line=dict(width=1, color='rgba(255,165,0,0.1)'),
                    showlegend=False
                ))
        # 2. Add 90% CI
        fig.add_trace(go.Scatter(
            x=list(range(forecast_days+1)),
            y=result['ci_90_upper'],
            line=dict(width=0.5, color='rgba(100,200,255,0.5)'),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=list(range(forecast_days+1)),
            y=result['ci_90_lower'],
            fill='tonexty',
            fillcolor='rgba(100,200,255,0.2)',
            line=dict(width=0.5, color='rgba(100,200,255,0.5)'),
            name='90% CI'
        ))

        # 3. Add 50% CI
        fig.add_trace(go.Scatter(
            x=list(range(forecast_days+1)),
            y=result['ci_50_upper'],
            line=dict(width=0),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=list(range(forecast_days+1)),
            y=result['ci_50_lower'],
            fill='tonexty',
            fillcolor='rgba(50,150,255,0.3)',
            line=dict(width=0),
            name='50% CI'
        ))

        # 4. Add mean line (top layer)
        fig.add_trace(go.Scatter(
            x=list(range(forecast_days+1)),
            y=result['mean'],
            line=dict(color='red', width=3),
            name='Mean Forecast'
        ))

        fig.update_layout(
            title=f"Dự báo giá {mc_stock} trong {forecast_days} ngày",
            xaxis_title='Ngày',
            yaxis_title='Giá (ngàn VND)',
            template='plotly_dark'
        )

        st.plotly_chart(fig, use_container_width=True)

        # Hiển thị thống kê
        # ===== PHẦN HIỂN THỊ THỐNG KÊ QUAN TRỌNG =====
        st.markdown("---")
        st.subheader("📊 Forecast Statistics")

        current_price = result['last_price']
        predicted_price = result['mean'][-1]
        # Tạo 3 cột
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Giá hiện tại", f"{current_price:,.0f} ngàn VND")
        with col2:
            st.metric(
                "Giá dự báo trung bình",
                f"{predicted_price:,.0f} ngàn VND",
                delta=f"{(predicted_price - current_price)/current_price:.2%}"
            )
        with col3:
            volatility = (result['ci_90_upper'][-1] - result['ci_90_lower'][-1]) / current_price
            st.metric("Biến động dự kiến", f"{volatility:.2%}")

            # Bảng thống kê chi tiết
        stats = pd.DataFrame({
            'Chỉ số': ['Trung bình', '5%', '25%', '75%', '95%'],
            'Giá (ngàn VND)': [
                result['mean'][-1],
                result['ci_90_lower'][-1],
                result['ci_50_lower'][-1],
                result['ci_50_upper'][-1],
                result['ci_90_upper'][-1]
            ]
        })

        st.dataframe(
            stats.style.format({'Giá (ngàn VND)': '{:,.0f}'}),
            hide_index=True,
            use_container_width=True
        )

if __name__ == "__main__":
    main()

Writing app.py


# WEB OUTPUT

In [3]:
!curl https://loca.lt/mytunnelpassword

35.234.60.95

In [4]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏your url is: https://slick-frogs-marry.loca.lt
